In [44]:
# Dependencies
from bs4 import BeautifulSoup as soup
import requests
import pymongo

In [45]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [46]:
# Define database and collection
db = client.mars_db
collection = db.mars

In [47]:
# URL of page to be scraped
mars_news_url = 'https://redplanetscience.com/'

# Retrieve page with the requests module
response = requests.get(mars_news_url)
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'lxml')

In [48]:
# retrieve the most recent news item
news_item = soup.find_all('div', class_='list_text')

for news in news_item:
    # Error handling
    try:
        # Retrieve the most recent news item: title
        news_title = soup.find('div', class_='content_title').text
        # date
        news_date = soup.find('div', class_='list_date').text
        # blurb
        news_blurb = soup.find('div', class_='article_teaser_body').text
        
        # Run only if title, price, and link are available
        if (news_title and news_date and news_blurb):
            # Print results
            print(news_title)
            print(news_date)
            print('-------------')
            print(news_blurb)
            
             # Dictionary to be inserted as a MongoDB document
            news_post = {
                'news_title': news_title,
                'news_date': news_date,
                'news_blurb': news_blurb
            }
            
            collection.insert_one(news_post)

    except Exception as e:
        print(e)

In [49]:
# Display items in MongoDB collection
news_stories = db.mars.find()

for news_story in news_stories:
    print(news_story)